In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np

In [ ]:
def full_truncation(S0, V0, rho, T, alpha, beta, gamma, r, M):
    np.random.seed(1234)
    N = 1_000
    dt = T / M
    S = np.zeros((N, M+1))
    S[:, 0] = S0
    V
    
    for i in range(1, M+1):
        z1 = np.random.randn(N)
        z2 = np.random.randn(N)
        dW1 = np.sqrt(dt) * z1
        dW2 = np.sqrt(dt) * (rho * z1 + np.sqrt(1 - rho**2) * z2)
        
        S[:, i] = S[:, i-1] * (1 + r * dt + np.sqrt(np.maximum(0, V)) * dW1)
        V = V + (alpha + beta * np.maximum(0, V)) * dt + gamma * np.sqrt(np.maximum(0, V)) * dW2
    
    A = S.mean(axis=1)
    price = np.exp(-r * T) * np.mean(np.maximum(S[:, -1] - A, 0))
    
    return price

price = full_truncation(S0=20, V0=0.06, rho=-0.6, T=2, alpha=0.45, beta=-5.105, gamma=0.25, r=0.05, M=100)
print("Price of the Asian option:", price)


Price of the Asian option: 2.402539589554329


Euler’s discretization scheme: $$ \tilde{X}_{t_{k+1}} = \tilde{X}_{t_k} + a(\tilde{X}_{t_k}) \Delta + b(\tilde{X}_{t_k}) \Delta W_{t_k} $$ 
For stock follow GBM: $$  \tilde{X}_{t_{k+1}} = \tilde{X}_{t_k} + r\tilde{X}_{t_k} \Delta + \sigma \tilde{X}_{t_k} \Delta W_{t_k} $$

In [24]:
# Monte Carlo European option(Euler’s discretization scheme)
def Euler_Option(S0, T, X, r, sigma, N, type):
    dt = T/N
    np.random.seed(123)
    St = S0 
    for i in range(int(N)):
        # Lecture 3 page 24 Euler decomposition
        St = St * (1 + r * dt + sigma * np.sqrt(dt) * np.random.randn(N))
    if type == 'call':
        price = np.exp(-r*T)*np.mean(np.maximum(St - X, 0))
    else:
        price = np.exp(-r*T)*np.mean(np.maximum(X - St, 0))
    # std = np.std(np.exp(-r*T)*np.maximum(St - X, 0))/np.sqrt(N-1)
    return price
Euler_Option(S0=100, T=5, X=100, r=0.055, sigma=0.2, N=1000, type = 'call')

31.56326385992646

Milshtein discretization scheme: $$ \tilde{X}_{t_{k+1}} = \tilde{X}_{t_k} + a(\tilde{X}_{t_k}) \Delta + b(\tilde{X}_{t_k}) \Delta W_{t_k} + \frac{1}{2} b(\tilde{X}_{t_k}) b'(\tilde{X}_{t_k}) \left\{ (\Delta W_{t_k})^2 - \Delta \right\} $$ 
For stock follow GBM: $$  \tilde{X}_{t_{k+1}} = \tilde{X}_{t_k} + r\tilde{X}_{t_k} \Delta + \sigma \tilde{X}_{t_k} \Delta W_{t_k} + \frac{1}{2} \sigma^2 \tilde{X}_{t_k} \left\{ (\Delta W_{t_k})^2 - \Delta \right\}$$


In [7]:
# Monte Carlo European option(Milshtein discretization scheme)
def Milshtein_option(S0, T, X, r, sigma, N, type):
    dt = T/N
    St = S0
    for i in range(N):
        delta_W = np.sqrt(dt) * np.random.randn(N)
        St = St * (1 + r * dt + sigma * delta_W + 0.5 * sigma**2 * ((delta_W)**2 - dt))

    if type == 'call':
        price = np.exp(-r*T)*np.mean(np.maximum(St - X, 0))
    else:
        price = np.exp(-r*T)*np.mean(np.maximum(X - St, 0))
    # std = np.std(np.exp(-r*T)*np.maximum(result - X, 0))/np.sqrt(N-1)
    return price
Milshtein_option(S0=100, T=5, X=100, r=0.055, sigma=0.2, N=1000, type = 'call')

In [9]:
# Approximate Normal Distribution
def N_cdf(x):
    d = [1, 0.0498673470, 0.0211410061, 0.0032776263, 0.0000380036, 0.0000488906,0.0000053830]
    if x >= 0:
        x_list = [1, x, x**2, x**3, x**4, x**5, x**6]
        return 1 - 0.5 * np.dot(x_list, d) ** (-16)
    else:
        return 1- N_cdf(-x)

In [15]:
# European option using BS with N approximation
def BSM_App(S0,K,T,t,sigma,r,y,type):
    phi = -1
    if type == "call":
        phi = 1
    z_p = (np.log(S0/K) + (r-y)*(T-t))/(sigma * np.sqrt(T-t)) + (sigma * np.sqrt(T-t))/2
    z_n = (np.log(S0/K) + (r-y)*(T-t))/(sigma * np.sqrt(T-t)) - (sigma * np.sqrt(T-t))/2    
    
    price = phi *(S0 * np.exp(-y * (T-t))* N_cdf(phi * z_p) - K * np.exp(-r * (T-t)) * N_cdf(phi * z_n))
    return price

BSM_App(S0=100,K=100,T=5,t=0,sigma=0.2,r=0.055,y=0,type='call')

30.372636979431498

In [26]:
# standard BSM model 
def BSM(S0,K,T,t,sigma,r,y,type):
    phi = -1
    if type == "call":
        phi = 1
    z_p = (np.log(S0/K) + (r-y)*(T-t))/(sigma * np.sqrt(T-t)) + (sigma * np.sqrt(T-t))/2
    z_n = (np.log(S0/K) + (r-y)*(T-t))/(sigma * np.sqrt(T-t)) - (sigma * np.sqrt(T-t))/2    
    
    price = phi *(S0 * np.exp(-y * (T-t))* norm.cdf(phi * z_p) - K * np.exp(-r * (T-t)) *norm.cdf(phi * z_n))
    return price

In [36]:
# Calculate Option Greeks
S0 = np.arange(95,106)
K = 100 
T = 0.5
t=0
sigma = 0.25 
r = 0.055
y = 0
epsilon = 0.00001
delta = (BSM(S0+epsilon,K,T,t,sigma,r,y,type='call') - BSM(S0-epsilon,K,T,t,sigma,r,y,type='call'))/ (2*epsilon)
gamma = (BSM(S0+epsilon,K,T,t,sigma,r,y,type='call') -  2 * BSM(S0,K,T,t,sigma,r,y,type='call') +\
          BSM(S0-epsilon,K,T,t,sigma,r,y,type='call'))/ ((epsilon)**2)
theta = (BSM(S0,K,T+epsilon,t,sigma,r,y,type='call') -  BSM(S0,K,T-epsilon,t,sigma,r,y,type='call'))/ (2*epsilon)
vega = (BSM(S0,K,T,t,sigma+epsilon,r,y,type='call') -  BSM(S0,K,T,t,sigma-epsilon,r,y,type='call'))/ (2*epsilon)

In [26]:
def full_truncation(S0, V0, rho, T, alpha, beta, gamma, r, K, M):
    np.random.seed(1234)
    N = 1_000
    V = V0
    dt = T/M
    S = np.zeros((N,M+1))
    # Reflection 
    S[:,0] = S0
    for i in range(1,M+1):
        z1 = np.random.randn(N)
        z2 = np.random.randn(N)
        dW1 = z1
        dW2 = rho * z1 + np.sqrt(1-rho**2) * z2
        S[:,i] = S[:,i-1] * (1 + r * dt + np.sqrt(np.maximum(0,V)) * np.sqrt(dt) * dW1)
        V = V + (alpha + (beta * np.maximum(0,V))) * dt + gamma * np.sqrt(np.maximum(0,V)) * np.sqrt(dt) * dW2
    price = np.exp(-r*T) * np.mean(np.maximum(S[:-1] - K, 0))  
    return price

full_truncation(S0=100, V0=0.05, rho=-0.6, T=5, alpha=0.45, beta=0.0625, gamma = 0.25, r=0.055, K=100, M=100)


40.37805326354636

In [105]:
# Estimation of Intergal using Halton sequence
def get_halton(N, base):
    seq = np.zeros(N)
    num_bits = 1 + int(np.ceil(np.log(N) / np.log(base)))
    vet_base = float(base)**-np.arange(1, num_bits + 1)
    work_vet = np.zeros(num_bits)

    for i in range(N):
        ok = 0
        j = 0
        while ok == 0:
            work_vet[j] += 1
            if work_vet[j] < base:
                ok = 1
            else:
                work_vet[j] = 0
                j += 1
        seq[i] = np.dot(work_vet, vet_base)

    return seq

N = 10_000
x = get_halton(N, 2)
y = get_halton(N, 3)

# Integral for estimation
result = (np.exp(-x * y) * (np.sin(6 * np.pi * x) + np.cbrt(np.cos(2 * np.pi * y)))).mean()
result

0.026261973509314397